#1. Исследовательский анализ данных

##На каких языках написаны статьи

Научные статьи могут быть написаны на различных языках, среди которых наиболее распространены:


* Английский

* Немецкий

* Французский

* Испанский

* Португальский

* Русский

Английский язык является доминирующим, большинство статей публикуется именно на нем.

##Тематики статей

Тематики научных статей могут варьироваться в зависимости от области исследования. Вот несколько основных категорий:

* Медицинские науки (фундаментальная медицина, биотехнологии в медицине)

* Гуманитарные науки (история, археология, философия)

* Естественные и точные науки (физика, химия, математика)

* Социальные науки (психология, экономика, политология)

* Инженерия и технологии (информационные технологии, машиностроение, энергетика)

##Проблемы в данных

При анализе научных статей могут возникнуть следующие проблемы:

* Устаревшие данные - статьи могут содержать нерелевантную и неактуальную информацию

* Отсутствие единого формата - разные журналы и издательства могут иметь различные требования к структуре и оформлению статей, а также сами статьи могут быть различного формата (pdf, doc, txt)

* Языковой барьер - статьи на разных языках могут быть сложными для анализа из-за различий в терминологии и написании

* Дубликаты - в статьях могут содержаться дубликаты, что также искажает результаты анализа

* Неполнота метаданных - отсутствие каких-либо данных в статье может негативно повлиять на анализ

#2. Выбор метрик и их обоснование

## Rouge
####Recall-Oriented Understudy for Gisting Evaluation

**ROUGE** — это набор метрик, который измеряет совпадения между резюме и оригинальным текстом. Он включает в себя:

* ROUGE-N (частота последовательности из N слов)
 * Rouge-1 (сравнивает однословные последовательности)
 * Rouge-2 (сравнивает двухсловные последовательности)

* ROUGE-L (сравнивает самые длинные общие подпоследовательности слов)

##BERTScore

**BERTScore** - использует предобученную модель BERT для оценки смыслового сходства резюме и исходного текста. Эта метрчка включает в себя:
* Precision (Точность) - сколько правильных ответов среди всех выданных ответов
* Recall (Полнота) - показывает долю правильных ответов
* F-score - среднее гармоническое между precision и recall

#3. Анализ моделей для резюмирования статей

###Установка библиотек

In [49]:
pip install pandas gigachain-community transformers torch rouge bert-score datasets

###Импортирование библиотек

In [50]:
import pandas as pd
from langchain.schema import HumanMessage, SystemMessage
from langchain_community.chat_models.gigachat import GigaChat
from transformers import T5Tokenizer, T5ForConditionalGeneration
import torch

###Загрузка данных

In [51]:
from datasets import load_dataset

# Загрузка датасета PubMed
ds = load_dataset("ccdv/pubmed-summarization", "section")

# Преобразование в pandas DataFrame
df = ds['train'].to_pandas()[:100]
# Так как на весь массив данных бесплатных токенов GigaChat не хватит, используем лишь первые 100 строк 
# В дальнейшем, планируется, что финансовые возможности позволят использовать GigaChat, не испытывая нехватку токенов

# Просмотр первых нескольких строк DataFrame
print(df.head())

                                             article  \
0  a recent systematic analysis showed that in 20...   
1  it occurs in more than 50% of patients and may...   
2  tardive dystonia ( td ) , a rarer side effect ...   
3  lepidoptera include agricultural pests that , ...   
4  syncope is caused by transient diffuse cerebra...   

                                            abstract  
0  background : the present study was carried out...  
1  backgroundanemia in patients with cancer who a...  
2  tardive dystonia ( td ) is a serious side effe...  
3  many lepidopteran insects are agricultural pes...  
4  we present an unusual case of recurrent cough ...  


###Резюмируем статьи с помощью моделей T5 и GigaChat

Запрашиваем ключ авторизации GigaChat API

In [52]:
Authorization_key = input()

ZTc0NTdhMjYtYjk5OC00MzE3LTllMzYtMjNkNGU2ZGY2NTRjOmEwZWZlNzRjLTAxYzUtNGQ5ZS1hNjllLWEyMTcwODJmZWMyMQ==


Инициализируем модели T5 и GigaChat

In [53]:
# Инициализация модели T5
t5_model_name = 't5-large'
t5_tokenizer = T5Tokenizer.from_pretrained(t5_model_name)
t5_model = T5ForConditionalGeneration.from_pretrained(t5_model_name)

# Инициализация модели GigaChat
chat = GigaChat(credentials=Authorization_key, scope='GIGACHAT_API_PERS', model='GigaChat', streaming=True, verify_ssl_certs=False)

Создание столбца с резюме при помощи GigaChat

In [54]:
# Функция для генерации резюме с помощью GigaChat
def summarize_with_gigachat(article):
    messages = [
                SystemMessage(content="You are an assistant for summarizing the text. Write an answer, depending on the language of the article itself. Use from 100 to 250 words and be laconic in the answer."),
                HumanMessage(content=article)
            ]
    return chat(messages).content

# Генерация резюме для GigaChat
df['gigachat_summary'] = df['article'].apply(summarize_with_gigachat)

Создание столбца с резюме при помощи T5

In [55]:
# Функция для генерации резюме с помощью T5
def summarize_with_t5(article):
    inputs = t5_tokenizer.encode("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
    summary_ids = t5_model.generate(inputs, max_length=250, min_length=100, early_stopping=True)
    return t5_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# Генерация резюме для T5
df['t5_summary'] = df['article'].apply(summarize_with_t5)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:615: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Оценка качества резюме и работы моделей

In [57]:
# Импортирование метрик
from rouge import Rouge
from bert_score import score

# Создание списка с аннотациями для метрики Bert score
references = df['abstract'].tolist()

# Подсчет метрик

# BERTScore
bert_scores_gigachat = []
bert_scores_t5 = []
for i in range(len(df['abstract'])):
    score_result_gigachat = score([df['gigachat_summary'][i]], [references[i]], lang='en')
    score_result_t5 = score([df['t5_summary'][i]], [references[i]], lang='en')
    bert_scores_gigachat.append(score_result_gigachat)
    bert_scores_t5.append(score_result_t5)

# Rouge-L score
rouge_l_scores_gigachat = []
rouge_l_scores_t5 = []

for index, row in df.iterrows():
    reference_summary = row['abstract']
    rouge = Rouge()

    # GigaChat
    predicted_summary_gigachat = row['gigachat_summary']
    rouge_scores_gigachat = rouge.get_scores(predicted_summary_gigachat, reference_summary, avg=True)
    rouge_l_scores_gigachat.append(rouge_scores_gigachat['rouge-l']['f'])

    # T5
    predicted_summary_t5 = row['t5_summary']
    rouge_scores_t5 = rouge.get_scores(predicted_summary_t5, reference_summary, avg=True)
    rouge_l_scores_t5.append(rouge_scores_t5['rouge-l']['f'])

# Вычисление среднего значения метрик для GigaChat
rouge_l_gigachat = sum(rouge_l_scores_gigachat) / len(rouge_l_scores_gigachat)
bert_score_gigachat = sum([score[2].item() for score in bert_scores_gigachat]) / len(bert_scores_gigachat)

# Вычисление среднего значения метрик для T5
rouge_l_t5 = sum(rouge_l_scores_t5) / len(rouge_l_scores_t5)
bert_score_t5 = sum([score[2].item() for score in bert_scores_t5]) / len(bert_scores_t5)

# Выводим результаты метрик для GigaChat
print("Результаты GigaChat")
print("Метрика Rouge-L", rouge_l_gigachat) # 0,24
print("Метрика Bert score:", bert_score_gigachat) # 0,84

# Выводим результаты метрик для T5
print("Результаты T5")
print("Метрика Rouge-L", rouge_l_t5) # 0,22
print("Метрика Bert score:", bert_score_t5) # 0,82

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaMod

Результаты GigaChat
Метрика Rouge-L 0.24150578422009317
Метрика Bert score: 0.8408654729525248
Результаты T5
Метрика Rouge-L 0.2218384973377089
Метрика Bert score: 0.8247800568739573


##Вывод

По показателям Rouge-L и BERTScore модель GigaChat лучше, чем T5, также стоит учитывать, что при работе на русском языке, результаты модели GigaChat станут ещё выше